In [22]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split

pd.options.display.max_columns = 100

from sklearn.metrics import confusion_matrix, classification_report, f1_score, recall_score, precision_score, plot_confusion_matrix, plot_roc_curve, roc_curve

In [2]:
key = pd.read_csv('Search Keyword.csv')
key.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Search keyword      290 non-null    object 
 1   Campaign            259 non-null    object 
 2   Campaign type       259 non-null    object 
 3   Campaign subtype    259 non-null    object 
 4   Device              259 non-null    object 
 5   Currency            259 non-null    object 
 6   Impr. (Top) %       259 non-null    object 
 7   Clicks              259 non-null    float64
 8   Impressions         259 non-null    float64
 9   CTR                 259 non-null    object 
 10  Avg. CPC            259 non-null    float64
 11  Cost                259 non-null    float64
 12  Conversions         259 non-null    float64
 13  View-through Conv.  259 non-null    float64
 14  Cost / conv.        259 non-null    float64
 15  Conv. rate          259 non-null    object 
dtypes: float

In [3]:
key.isna().sum()

Search keyword         0
Campaign              31
Campaign type         31
Campaign subtype      31
Device                31
Currency              31
Impr. (Top) %         31
Clicks                31
Impressions           31
CTR                   31
Avg. CPC              31
Cost                  31
Conversions           31
View-through Conv.    31
Cost / conv.          31
Conv. rate            31
dtype: int64

In [4]:
key.dropna(inplace=True)

In [5]:
key.isna().sum()

Search keyword        0
Campaign              0
Campaign type         0
Campaign subtype      0
Device                0
Currency              0
Impr. (Top) %         0
Clicks                0
Impressions           0
CTR                   0
Avg. CPC              0
Cost                  0
Conversions           0
View-through Conv.    0
Cost / conv.          0
Conv. rate            0
dtype: int64

In [6]:
key.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 0 to 289
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Search keyword      259 non-null    object 
 1   Campaign            259 non-null    object 
 2   Campaign type       259 non-null    object 
 3   Campaign subtype    259 non-null    object 
 4   Device              259 non-null    object 
 5   Currency            259 non-null    object 
 6   Impr. (Top) %       259 non-null    object 
 7   Clicks              259 non-null    float64
 8   Impressions         259 non-null    float64
 9   CTR                 259 non-null    object 
 10  Avg. CPC            259 non-null    float64
 11  Cost                259 non-null    float64
 12  Conversions         259 non-null    float64
 13  View-through Conv.  259 non-null    float64
 14  Cost / conv.        259 non-null    float64
 15  Conv. rate          259 non-null    object 
dtypes: float

In [7]:
key.head()

,Search keyword,Campaign,Campaign type,Campaign subtype,Device,Currency,Impr. (Top) %,Clicks,Impressions,CTR,Avg. CPC,Cost,Conversions,View-through Conv.,Cost / conv.,Conv. rate
0,perumahan subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
1,rumah kpr bersubsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
2,rumah subsidi daerah serang,CPA Leads Experiment,Search,All features,TV screens,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
3,rumah subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
4,rumah subsidi daerah serang,CPA Leads Experiment,Search,All features,Desktop,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%


In [8]:
for i in key.columns:
    print(i)
    print(key[i].unique())
    print()

Search keyword
['perumahan subsidi' 'rumah kpr bersubsidi' 'rumah subsidi daerah serang'
 'rumah subsidi' '+rumah +subsidi +serang' 'rumah subsidi serang'
 'perumahan bersubsidi' 'rumah subsidi pemerintah' 'kpr bersubsidi'
 'perumahan subsidi serang' 'harga rumah subsidi daerah cilegon'
 'perumahan subsidi di serang banten 2019' 'rumah subsidi serang banten'
 '+kpr +subsidi +pemerintah' 'perumahan subsidi di serang banten 2020'
 'perumahan kpr bersubsidi' 'rumah kpr murah'
 'perumahan subsidi pemerintah' 'kpr rumah murah'
 '+rumah +subsidi +serang +2021' 'kredit rumah btn'
 'perumahan subsidi di kramatwatu serang' '+rumah +murah +bersubsidi'
 'perumahan subsidi di serang banten 2021'
 '+perumahan +subsidi + serang +banten' '+perumahan +subsidi +serang'
 '+rumah +subsidi +banten' 'rumah bersubsidi'
 '+perumahan +subsidi +pemerintah' '+rumah +murah +subsidi']

Campaign
['CPA Leads Experiment' 'rumahSubsidi // griyaBukitIntan' 'Campaign #1'
 'Promo // Griya Bukit Intan // 4 Juta']

Campai

In [9]:
key.columns = key.columns.str.lower()
key.head()

,search keyword,campaign,campaign type,campaign subtype,device,currency,impr. (top) %,clicks,impressions,ctr,avg. cpc,cost,conversions,view-through conv.,cost / conv.,conv. rate
0,perumahan subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
1,rumah kpr bersubsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
2,rumah subsidi daerah serang,CPA Leads Experiment,Search,All features,TV screens,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
3,rumah subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%
4,rumah subsidi daerah serang,CPA Leads Experiment,Search,All features,Desktop,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%


In [10]:
key.describe()

,clicks,impressions,avg. cpc,cost,conversions,view-through conv.,cost / conv.
count,259.000000,259.000000,259.000000,259.000000,259.0,259.0,259.0
mean,0.003861,0.328185,3.281853,3.281853,0.0,0.0,0.0
std,0.062137,2.233557,52.816430,52.816430,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
75%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
max,1.000000,30.000000,850.000000,850.000000,0.0,0.0,0.0


In [11]:
key['search keyword'] = key['search keyword'].str.replace('+','')

In [12]:
key['search keyword'].unique()

array(['perumahan subsidi', 'rumah kpr bersubsidi',
       'rumah subsidi daerah serang', 'rumah subsidi',
       'rumah subsidi serang', 'perumahan bersubsidi',
       'rumah subsidi pemerintah', 'kpr bersubsidi',
       'perumahan subsidi serang', 'harga rumah subsidi daerah cilegon',
       'perumahan subsidi di serang banten 2019',
       'rumah subsidi serang banten', 'kpr subsidi pemerintah',
       'perumahan subsidi di serang banten 2020',
       'perumahan kpr bersubsidi', 'rumah kpr murah',
       'perumahan subsidi pemerintah', 'kpr rumah murah',
       'rumah subsidi serang 2021', 'kredit rumah btn',
       'perumahan subsidi di kramatwatu serang', 'rumah murah bersubsidi',
       'perumahan subsidi di serang banten 2021',
       'perumahan subsidi  serang banten', 'rumah subsidi banten',
       'rumah bersubsidi', 'rumah murah subsidi'], dtype=object)

In [13]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
# print(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /Users/ndaru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
def remove_stopwords(sentence):
    stop_words = stopwords.words('indonesian')
    return ' '.join([w for w in nltk.word_tokenize(sentence) if not w in stop_words])

In [15]:
key['keyword cleaned'] = [remove_stopwords(i) for i in key['search keyword']]

In [16]:
key['keyword cleaned'].unique()

array(['perumahan subsidi', 'rumah kpr bersubsidi',
       'rumah subsidi daerah serang', 'rumah subsidi',
       'rumah subsidi serang', 'perumahan bersubsidi',
       'rumah subsidi pemerintah', 'kpr bersubsidi',
       'perumahan subsidi serang', 'harga rumah subsidi daerah cilegon',
       'perumahan subsidi serang banten 2019',
       'rumah subsidi serang banten', 'kpr subsidi pemerintah',
       'perumahan subsidi serang banten 2020', 'perumahan kpr bersubsidi',
       'rumah kpr murah', 'perumahan subsidi pemerintah',
       'kpr rumah murah', 'rumah subsidi serang 2021', 'kredit rumah btn',
       'perumahan subsidi kramatwatu serang', 'rumah murah bersubsidi',
       'perumahan subsidi serang banten 2021',
       'perumahan subsidi serang banten', 'rumah subsidi banten',
       'rumah bersubsidi', 'rumah murah subsidi'], dtype=object)

In [17]:
key['keyword cleaned'] = key['keyword cleaned'].str.split(' ')
key['keyword cleaned'] = key['keyword cleaned'].apply(', '.join)
key['keyword cleaned'].to_frame()

,keyword cleaned
0,"perumahan, subsidi"
1,"rumah, kpr, bersubsidi"
2,"rumah, subsidi, daerah, serang"
3,"rumah, subsidi"
4,"rumah, subsidi, daerah, serang"
...,...
284,"perumahan, subsidi, serang, banten, 2021"
285,"kpr, bersubsidi"
286,"rumah, subsidi, pemerintah"
288,"rumah, subsidi"


In [18]:
cvr = CountVectorizer(
    tokenizer = lambda x:x.split(',')
)

keywords = cvr.fit_transform(key['keyword cleaned'])
keywords.toarray()

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [19]:
print(cvr.get_feature_names())

[' 2019', ' 2020', ' 2021', ' banten', ' bersubsidi', ' btn', ' cilegon', ' daerah', ' kpr', ' kramatwatu', ' murah', ' pemerintah', ' rumah', ' serang', ' subsidi', 'harga', 'kpr', 'kredit', 'perumahan', 'rumah']


In [23]:
cos_score = cosine_similarity(keywords)
cos_score

array([[1.        , 0.        , 0.35355339, ..., 0.40824829, 0.5       ,
        0.5       ],
       [0.        , 1.        , 0.28867513, ..., 0.33333333, 0.40824829,
        0.40824829],
       [0.35355339, 0.28867513, 1.        , ..., 0.57735027, 0.70710678,
        0.        ],
       ...,
       [0.40824829, 0.33333333, 0.57735027, ..., 1.        , 0.81649658,
        0.        ],
       [0.5       , 0.40824829, 0.70710678, ..., 0.81649658, 1.        ,
        0.        ],
       [0.5       , 0.40824829, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [24]:
pd.DataFrame(cos_score, index = key['campaign'], columns=key['campaign'])

campaign,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,Campaign #1,Promo // Griya Bukit Intan // 4 Juta,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,Promo // Griya Bukit Intan // 4 Juta,Campaign #1,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,Promo // Griya Bukit Intan // 4 Juta,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,CPA Leads Experiment,CPA Leads Experiment,...,rumahSubsidi // griyaBukitIntan,Promo // Griya Bukit Intan // 4 Juta,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,Promo // Griya Bukit Intan // 4 Juta,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,Promo // Griya Bukit Intan // 4 Juta,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,Promo // Griya Bukit Intan // 4 Juta,CPA Leads Experiment,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,rumahSubsidi // griyaBukitIntan,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment,CPA Leads Experiment
campaign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CPA Leads Experiment,1.000000,0.000000,0.353553,0.500000,0.353553,0.408248,0.408248,0.500000,0.408248,0.408248,0.000000,0.500000,0.816497,0.408248,0.316228,0.632456,0.500000,0.316228,0.408248,1.000000,0.353553,0.408248,0.500000,0.632456,0.408248,0.000000,0.408248,0.353553,0.316228,0.408248,0.816497,0.000000,1.000000,0.500000,0.500000,0.408248,0.408248,0.816497,1.000000,0.353553,0.408248,0.0,0.707107,0.000000,0.632456,0.707107,0.500000,0.353553,0.500000,0.000000,...,0.408248,0.408248,0.000000,0.0,0.816497,0.707107,0.000000,0.353553,0.000000,0.000000,0.707107,0.000000,0.500000,0.000000,0.632456,0.816497,0.353553,0.816497,0.000000,0.000000,0.408248,0.000000,0.000000,0.408248,0.408248,0.000000,0.000000,0.408248,1.000000,0.632456,0.632456,0.000000,1.000000,0.408248,0.632456,0.000000,0.000000,0.500000,0.500000,0.353553,0.408248,0.500000,0.0,1.000000,0.353553,0.632456,0.000000,0.408248,0.500000,0.500000
CPA Leads Experiment,0.000000,1.000000,0.288675,0.408248,0.288675,0.333333,0.333333,0.408248,0.333333,0.333333,0.408248,0.408248,0.000000,0.333333,0.000000,0.000000,0.408248,0.000000,0.333333,0.000000,0.288675,0.000000,0.40

In [25]:
product_like = input('Which campaign that is similar?')
index_like = key[key['campaign']== product_like].index[0]
campaign_recom = list(enumerate(cos_score[index_like]))

Which campaign that is similar? CPA Leads Experiment


In [29]:
campaign_recom_80 = list(filter(lambda x: x[1] > 0.8, campaign_recom))
campaign_recom_80_sorted = sorted(campaign_recom_80, key=lambda x: x[1], reverse=True)
campaign_recom_80_sorted 

[(0, 0.9999999999999998),
 (19, 0.9999999999999998),
 (32, 0.9999999999999998),
 (38, 0.9999999999999998),
 (62, 0.9999999999999998),
 (84, 0.9999999999999998),
 (90, 0.9999999999999998),
 (106, 0.9999999999999998),
 (127, 0.9999999999999998),
 (137, 0.9999999999999998),
 (161, 0.9999999999999998),
 (190, 0.9999999999999998),
 (196, 0.9999999999999998),
 (200, 0.9999999999999998),
 (237, 0.9999999999999998),
 (241, 0.9999999999999998),
 (252, 0.9999999999999998),
 (12, 0.816496580927726),
 (30, 0.816496580927726),
 (37, 0.816496580927726),
 (51, 0.816496580927726),
 (64, 0.816496580927726),
 (66, 0.816496580927726),
 (68, 0.816496580927726),
 (71, 0.816496580927726),
 (80, 0.816496580927726),
 (81, 0.816496580927726),
 (100, 0.816496580927726),
 (101, 0.816496580927726),
 (104, 0.816496580927726),
 (105, 0.816496580927726),
 (110, 0.816496580927726),
 (126, 0.816496580927726),
 (145, 0.816496580927726),
 (146, 0.816496580927726),
 (151, 0.816496580927726),
 (152, 0.816496580927726),
 (

In [30]:
for i in campaign_recom_70_sorted[1:]:
    print(key.iloc[i[0]]['campaign'])

rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
CPA Leads Experiment
CPA Leads Experiment
CPA Leads Experiment
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
CPA Leads Experiment
Promo // Griya Bukit Intan // 4 Juta
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
Promo // Griya Bukit Intan // 4 Juta
Promo // Griya Bukit Intan // 4 Juta
Promo // Griya Bukit Intan // 4 Juta
CPA Leads Experiment
CPA Leads Experiment
Promo // Griya Bukit Intan // 4 Juta
rumahSubsidi // griyaBukitIntan
CPA Leads Experiment
CPA Leads Experiment
Promo // Griya Bukit Intan // 4 Juta
Prom

In [39]:
print(len(key[key['impressions']>0]))
no_impr = key[key['impressions']>0]
no_impr.sort_values('impressions', ascending=False)

24


,search keyword,campaign,campaign type,campaign subtype,device,currency,impr. (top) %,clicks,impressions,ctr,avg. cpc,cost,conversions,view-through conv.,cost / conv.,conv. rate,keyword cleaned
110,rumah subsidi,CPA Leads Experiment,Search,All features,Desktop,IDR,25.00%,0.0,30.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
12,rumah subsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Desktop,IDR,35.71%,0.0,17.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
242,rumah bersubsidi,CPA Leads Experiment,Search,All features,Mobile,IDR,100.00%,1.0,8.0,12.50%,850.0,850.0,0.0,0.0,0.0,0.00%,"rumah, bersubsidi"
226,perumahan subsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Desktop,IDR,100.00%,0.0,5.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"perumahan, subsidi"
107,kpr subsidi pemerintah,Promo // Griya Bukit Intan // 4 Juta,Search,All features,Desktop,IDR,100.00%,0.0,3.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"kpr, subsidi, pemerintah"
259,rumah kpr bersubsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Desktop,IDR,100.00%,0.0,2.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, kpr, bersubsidi"
91,perumahan subsidi di serang banten 2020,CPA Leads Experiment,Search,All features,Desktop,IDR,0.00%,0.0,2.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"perumahan, subsidi, serang, banten, 2020"
202,kredit rumah btn,rumahSubsidi // griyaBukitIntan,Search,All features,Mobile,IDR,100.00%,0.0,2.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"kredit, rumah, btn"
212,perumahan bersubsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Mobile,IDR,100.00%,0.0,1.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"perumahan, bersubsidi"
270,rumah subsidi serang,rumahSubsidi // griyaBukitIntan,Search,All features,Mobile,IDR,0.00%,0.0,1.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi, serang"


In [38]:
print(len(no_impr['keyword cleaned'].unique()))
no_impr['keyword cleaned'].unique()

16


array(['rumah, subsidi', 'perumahan, kpr, bersubsidi',
       'perumahan, subsidi', 'rumah, subsidi, serang',
       'perumahan, bersubsidi', 'rumah, murah, bersubsidi',
       'perumahan, subsidi, serang, banten, 2020', 'kredit, rumah, btn',
       'kpr, subsidi, pemerintah', 'kpr, rumah, murah',
       'rumah, subsidi, pemerintah', 'rumah, subsidi, banten',
       'perumahan, subsidi, serang', 'rumah, bersubsidi',
       'rumah, kpr, bersubsidi', 'kpr, bersubsidi'], dtype=object)

In [40]:
key[key['search keyword']=='rumah subsidi']

,search keyword,campaign,campaign type,campaign subtype,device,currency,impr. (top) %,clicks,impressions,ctr,avg. cpc,cost,conversions,view-through conv.,cost / conv.,conv. rate,keyword cleaned
3,rumah subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
12,rumah subsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Desktop,IDR,35.71%,0.0,17.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
37,rumah subsidi,rumahSubsidi // griyaBukitIntan,Search,All features,Tablet,IDR,100.00%,0.0,1.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
39,rumah subsidi,CPA Leads Experiment,Search,All features,Tablet,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
63,rumah subsidi,CPA Leads Experiment,Search,All features,Mobile,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
86,rumah subsidi,CPA Leads Experiment,Search,All features,Other,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
110,rumah subsidi,CPA Leads Experiment,Search,All features,Desktop,IDR,25.00%,0.0,30.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
146,rumah subsidi,rumahSubsidi // griyaBukitIntan,Search,All features,TV screens,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
171,rumah subsidi,CPA Leads Experiment,Search,All features,Desktop,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
184,rumah subsidi,CPA Leads Experiment,Search,All features,Other,IDR,0.00%,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.00%,"rumah, subsidi"
